In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## API로 산업군 부여

In [ ]:
!pip install openai==0.28
import pandas as pd
import openai

# OpenAI API 키 설정
openai.api_key = 'sk-'

In [ ]:
def classify_company(company_name):
    sectors = [
        "건설",
        "중공업",
        "철강/소재",
        "에너지/화학",
        "정보기술",
        "금융",
        "생활소비재",
        "경기소비재",
        "산업재",
        "헬스케어"
    ]

    # 대화형 API 요청을 위한 prompt 설정
    prompt = f"Given the company name '{company_name}', classify it into one of the following sectors: {', '.join(sectors)}. 반환 결과는 문장으로 출력하지 말고, 각 산업군명으로만 부여하십시오. 종합 기업의 경우 비중이 큰 사업 분야에 해당하는 분야로 분류하십시오."

    # 대화형 세션 생성
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # 최신 모델 사용
        messages=[
            {"role": "system", "content": "You are a smart assistant."},
            {"role": "user", "content": prompt}
        ]
    )

    # 수정된 응답 처리 방식
    return response.choices[0].message['content'].strip()

In [ ]:
# 'industry' 컬럼을 추가하고, 각 기업명에 대해 분류를 수행하여 결과를 저장
for index, row in data.iterrows():
    company_name = row['company']
    sector = classify_company(company_name)
    data.at[index, 'industry'] = sector

# 결과를 새 CSV 파일로 저장
output_file_path = '/content/drive/MyDrive/코스피200기사수_정렬_섹터10개로 분류.csv'
data.to_csv(output_file_path, index=False)

print(f"분류가 완료된 파일이 '{output_file_path}'에 저장되었습니다.")

분류가 완료된 파일이 '/content/drive/MyDrive/코스피200기사수_정렬_섹터10개로 분류.csv'에 저장되었습니다.


## 로우데이터 전처리 및 통합, 산업군 재분류

In [3]:
import pandas as pd

In [4]:
# CSV 파일 로드
file_path = '/content/drive/MyDrive/docs company_분류.csv'
df1 = pd.read_csv(file_path)
# CSV 파일 로드
file_path = '/content/drive/MyDrive/코스피200기사수_정렬_섹터10개로 분류.csv'
df2 = pd.read_csv(file_path)
# CSV 파일 로드
file_path = '/content/drive/MyDrive/data_0536_20240330.csv'
df3 = pd.read_csv(file_path, encoding='euc-kr')

In [5]:
df1.head()

,company,industry
0,S-Oil,에너지/화학
1,포스코홀딩스,철강/소재
2,삼성물산,생활소비재
3,현대차,산업재
4,CJ제일제당,생활소비재


In [6]:
df2.head()
# 결과 확인
#df2.iloc[0:1,1:2] = 253
#df2.iloc[0:1,:]

,company,count,industry
0,F&F,253,생활소비재
1,S-Oil,3060000,에너지/화학
2,LG,1680000,정보기술
3,SK,1580000,산업재
4,카카오,109000,정보기술


In [7]:
df3.head()

,종목코드,종목명,종가,대비,등락률,시가,고가,저가,거래량,거래대금,시가총액,상장주식수
0,095570,AJ네트웍스,4865,-255,-4.98,5140,5140,4860,200607,993838430,220154672535,45252759
1,006840,AK홀딩스,15100,-460,-2.96,15600,15600,14810,21354,321830250,200038171100,13247561
2,027410,BGF,3740,-55,-1.45,3795,3815,3730,101167,380557375,357980798340,95716791
3,282330,BGF리테일,117700,700,0.60,117600,118700,117000,60274,7080399100,2034315736200,17283906
4,138930,BNK금융지주,7770,-210,-2.63,8020,8020,7770,874160,6842692780,2502627163260,322088438


In [ ]:
# 'merged' 컬럼을 생성하여, df1에는 'O', df2에는 'N'을 기본값으로 할당합니다.
df1['merged'] = 'Legacy'  # Original
df2['merged'] = 'KOSPI 200'  # New

# df1과 df2를 다시 'company'와 'industry'를 기준으로 full outer join 수행
# 이번에는 정확한 'merged' 컬럼 값 할당을 위해 조건부 로직을 적용
merged_df_final = pd.merge(df1[['company', 'industry', 'merged']], df2[['company', 'count','industry']],
                           on=['company', 'industry'], how='outer', indicator=True)

merged_df_final

In [ ]:
# '_merge' 컬럼에 따라 'merged' 컬럼의 값을 설정
# 'left_only'는 df1에만 있는 데이터, 'both'는 양쪽 모두에 있는 데이터
merged_df_final['merged'] = merged_df_final['_merge'].apply(lambda x: 'Legacy' if x == 'left_only' else ('Merged' if x == 'both' else 'KOSPI 200'))

# '_merge' 컬럼을 제거
merged_df_final.drop('_merge', axis=1, inplace=True)
merged_df_final

In [10]:
# 'merged' 컬럼의 각각의 값에 대한 개수를 세기
merged_counts = merged_df_final['merged'].value_counts()

merged_counts

KOSPI 200    175
Legacy        42
Merged        25
Name: merged, dtype: int64

In [ ]:
# 기존 데이터프레임(merged_df)을 기준으로 하여 '시가총액' 컬럼을 합칩니다.
# 여기서는 'left' 합병을 사용하여 기존 데이터프레임에만 존재하는 행들을 유지합니다.
df3['company'] = df3['종목명']
final_df = pd.merge(merged_df_final, df3, on="company", how="left")
final_df2 = final_df[['merged','industry','종목코드','company','count','rank_in_industry','시가총액']]
# 산업군별 시가총액 순위 부여
final_df2['rank_in_fundmental'] = final_df2.groupby('industry')['시가총액'].rank(method='dense', ascending=True)
final_df2 = final_df2 .sort_values(by='시가총액', ascending=False)
final_df2

In [17]:
unique_ranks_per_industry = final_df2.groupby('industry')['rank_in_industry'].nunique()
unique_ranks_per_industry

industry
건설                  8
경기소비재               2
금융                 30
물류/운송               1
산업재                20
생명공학                1
생명과학                2
생명과학 healthcare     1
생활소비재              28
식품/음료               1
에너지/화학             15
운송/물류               2
자동차 및 운송장비산업.       1
전력/에너지              0
정보기술               21
제약/바이오              0
종합                  1
중공업                14
철강/소재              10
첨단소재: 철강/소재         1
항공/방위산업             1
해운운송                0
헬스케어                4
화학                  3
환경 및 에너지.           1
Name: rank_in_industry, dtype: int64

In [18]:
# 산업군을 대분류로 매핑하는 딕셔너리 생성
industry_mapping = {
    '에너지/화학': '에너지/화학', '전력/에너지': '에너지/화학', '화학': '에너지/화학','환경 및 에너지.': '에너지/화학',
    '철강/소재': '소재', '첨단소재: 철강/소재': '소재',
    '생활소비재': '소비재', '경기소비재': '소비재', '식품/음료': '소비재',
    '산업재': '산업', '자동차 및 운송장비산업.': '산업', '항공/방위산업': '산업',
    '중공업': '중공업',
    '생명과학': '헬스케어', '제약/바이오': '헬스케어', '헬스케어': '헬스케어', '생명공학': '헬스케어', '생명과학 healthcare': '헬스케어',
    '금융': '금융',
    '정보기술': '정보기술',
    '건설': '건설',
    '해운운송': '운송', '운송/물류': '운송', '물류/운송': '운송',
    '종합': '기타',
}

# 매핑 정보를 사용하여 새로운 대분류 컬럼 생성
final_df2['대분류'] = final_df2['industry'].map(industry_mapping)

In [22]:
companyconuts_per_대분류 = final_df2.groupby('대분류')['company'].nunique()
companyconuts_per_대분류

대분류
건설        14
금융        36
기타         1
산업        40
소비재       40
소재        18
에너지/화학    28
운송         4
정보기술      30
중공업       19
헬스케어      10
Name: company, dtype: int64

In [23]:
final_df2['company'].nunique()

232

In [ ]:
# 'merged' 컬럼이 'O'인 데이터만 필터링하여 보기
merged_o_only = merged_df_final[merged_df_final['merged'] == '']
#count_nan_only = merged_df_final[merged_df_final['count'].isna()]
merged_o_only


,company,count,industry,merged,rank_in_industry,종목코드,시가총액,상장주식수,rank_in_fundmental
1,포스코홀딩스,NaN,철강/소재,O,NaN,NaN,NaN,NaN,NaN
5,삼성바이오로직스,NaN,생명과학,O,NaN,207940,5.914559e+13,71174000.0,1.0
9,삼천리,NaN,건설,O,NaN,004690,3.803613e+11,4055025.0,13.0
10,GS에너지,NaN,에너지/화학,O,NaN,NaN,NaN,NaN,NaN
14,HD현대에너지솔루션,NaN,에너지/화학,O,NaN,322000,2.755200e+11,11200000.0,22.0
15,고려아연,NaN,철강/소재,O,NaN,NaN,NaN,NaN,NaN
16,코오롱,NaN,산업재,O,NaN,NaN,NaN,NaN,NaN
17,OCI,NaN,산업재,O,NaN,NaN,NaN,NaN,NaN
18,세아제강,NaN,철강/소재,O,NaN,306200,3.854532e+11,2836300.0,14.0
20,KCC,NaN,산업재,O,NaN,002380,2.306039e+12,8886471.0,12.0


In [ ]:
final_df2['industry'].unique()

array(['에너지/화학', '철강/소재', '생활소비재', '산업재', '생명과학', '금융', '정보기술', '건설',
       '중공업', '해운운송', '전력/에너지', '제약/바이오', '헬스케어', '생명공학', '자동차 및 운송장비산업.',
       '경기소비재', '항공/방위산업', '식품/음료', '운송/물류', '생명과학 healthcare', '물류/운송',
       '종합', '화학', '환경 및 에너지.', '첨단소재: 철강/소재'], dtype=object)

In [24]:
final_df2['대분류'].unique()

array(['정보기술', '에너지/화학', '헬스케어', '산업', '소재', '소비재', '금융', '중공업', '운송',
       '건설', '기타'], dtype=object)

In [26]:
# 각 대분류에 속하는 company의 개수를 계산
category_counts = final_df2['대분류'].value_counts()

# 결과 출력
print(category_counts)


산업        40
소비재       40
금융        36
정보기술      30
에너지/화학    28
중공업       19
소재        18
건설        14
헬스케어      12
운송         4
기타         1
Name: 대분류, dtype: int64


## 산업군별 시총분포, 기사발생량분포, 가중평균순위 산출

In [ ]:
# 중복 제거
# 카테고리 10개로 단일화
# 결측치 제거
# 산업군별 top 10개만 남기기


In [34]:
# CSV 파일 로드
file_path = '/content/KOSPI200_searchtime_and_fundmenmtal_level (2)_.csv'
df4 = pd.read_csv(file_path, encoding = 'euc-kr')

In [35]:
unique_ranks_per_industry = df4.groupby('대분류')['company'].nunique()
unique_ranks_per_industry

대분류
건설        11
금융        29
미디어        8
산업        24
소비재       37
소재        15
에너지/화학    28
운송         7
정보기술      16
중공업       16
지주사        9
헬스케어      10
Name: company, dtype: int64

In [36]:
# 각 산업군('industry') 내에서 'count' 컬럼에 따라 순위를 매기고, 결과를 새로운 컬럼에 저장합니다.
df4 ['산업군내기사발생량순위'] = df4.groupby('대분류')['article'].rank(method='dense', ascending=False)
df4 ['산업군내시총순위'] = df4.groupby('대분류')['fundment'].rank(method='dense', ascending=False)
# 결과의 첫 몇 줄을 출력하여 확인합니다.
df4

,merged,종목코드,article,fundment,company,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위
0,KOSPI 200,207940,685,59145594000000,삼성바이오로직스,헬스케어,헬스케어,7.0,1.0
1,Merged,128940,7530,4266060003000,한미약품,헬스케어,헬스케어,2.0,5.0
2,KOSPI 200,185750,2100,1451821084000,종근당,헬스케어,헬스케어,5.0,8.0
3,KOSPI 200,69620,2280,1405451547500,대웅제약,헬스케어,헬스케어,4.0,9.0
4,KOSPI 200,100,300,6152035208800,유한양행,헬스케어,생활소비재,8.0,3.0
...,...,...,...,...,...,...,...,...,...
205,KOSPI 200,47040,3890,1539881873790,대우건설,건설,건설,5.0,6.0
206,KOSPI 200,103140,533,1415226039000,풍산,건설,건설,6.0,7.0
207,KOSPI 200,6360,5220,1279443275500,GS건설,건설,건설,4.0,8.0
208,KOSPI 200,300720,300,851916942000,한일시멘트,건설,건설,7.0,10.0


### 산업군별 시총순위분포

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# 시총순위별로 그룹화하고, 각 그룹 내에서 대분류별로 시총의 평균을 계산
ranked_cap = df4.groupby(['산업군내시총순위', '대분류'])['fundment'].mean().unstack()

# 시각화
plt.figure(figsize=(12, 8))
for column in ranked_cap.columns:
    plt.plot(ranked_cap.index, ranked_cap[column], marker='o', label=column)

plt.title('대분류별 시총순위에 따른 평균 시총 비교')
plt.xlabel('시총순위')
plt.ylabel('평균 시총')
plt.legend(title='대분류')
plt.grid(True)
plt.show()


In [54]:
df6 = df4
df6 = df6[~((df6['대분류'] == '정보기술') & (df6['산업군내시총순위'] <= 2.0))]
df_filtered = df6[~((df6['대분류'] == '에너지/화학') & (df6['산업군내시총순위'] <= 1.0))]

In [ ]:
# 각 대분류별로 시총순위 15위까지만 필터링
df_filtered = df_filtered[df_filtered['산업군내시총순위'] <= 5]

# 시총순위별로 그룹화하고, 각 그룹 내에서 대분류별로 시총의 평균을 계산
ranked_cap = df_filtered.groupby(['산업군내시총순위', '대분류'])['fundment'].mean().unstack()

# 시각화
plt.figure(figsize=(14, 30))
for column in ranked_cap.columns:
    plt.plot(ranked_cap.index, ranked_cap[column], marker='o', label=column)
    for idx in ranked_cap.index:
        if pd.notna(ranked_cap.at[idx, column]):
            company_name = df_filtered[(df_filtered['대분류'] == column) & (df_filtered['산업군내시총순위'] == idx)]['company'].values[0]
            plt.text(idx, ranked_cap.at[idx, column], company_name, ha='center')

plt.title('대분류별 시총순위에 따른 평균 시총 비교 (정보기술 1위 제외)')
plt.xlabel('시총순위')
plt.ylabel('평균 시총')
plt.legend(title='대분류')
plt.grid(True)
plt.show()

### 산업군별 기사발생량순위분포

In [59]:
df6 = df4
df6 = df6[~((df6['대분류'] == '지주사') & (df6['산업군내기사발생량순위'] <= 5.0))]
df_filtered = df6[~((df6['대분류'] == '에너지/화학') & (df6['산업군내기사발생량순위'] <= 1.0))]

In [ ]:
# 각 대분류별로 시총순위 15위까지만 필터링
df_filtered = df_filtered[df_filtered['산업군내기사발생량순위'] <= 5]

# 시총순위별로 그룹화하고, 각 그룹 내에서 대분류별로 시총의 평균을 계산
ranked_cap = df_filtered.groupby(['산업군내기사발생량순위', '대분류'])['article'].mean().unstack()

# 시각화
plt.figure(figsize=(14, 30))
for column in ranked_cap.columns:
    plt.plot(ranked_cap.index, ranked_cap[column], marker='o', label=column)
    for idx in ranked_cap.index:
        if pd.notna(ranked_cap.at[idx, column]):
            company_name = df_filtered[(df_filtered['대분류'] == column) & (df_filtered['산업군내기사발생량순위'] == idx)]['company'].values[0]
            plt.text(idx, ranked_cap.at[idx, column], company_name, ha='center')

plt.title('대분류별 시총순위에 따른 평균 시총 비교 (정보기술 1위 제외)')
plt.xlabel('시총순위')
plt.ylabel('평균 시총')
plt.legend(title='대분류')
plt.grid(True)
plt.show()

### 가중평균순위 산출

In [67]:
import pandas as pd

df7 = df4

# 가중치 설정
시총순위_가중치 = 0.5
기사발생량순위_가중치 = 0.5

# 가중평균 계산
df7['가중평균순위'] = (df7['산업군내시총순위'] * 시총순위_가중치 +
                 df7['산업군내기사발생량순위'] * 기사발생량순위_가중치)

# 새로운 순위에 따라 DataFrame 정렬 (선택적)
df7 = df7.sort_values('가중평균순위')

df7

,merged,종목코드,article,fundment,company,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위
13,KOSPI 200,3550,1680000,13763836887500,LG,지주사,정보기술,1.0,1.0,1.0
200,KOSPI 200,720,39900,3674740245000,현대건설,건설,건설,1.0,1.0,1.0
56,KOSPI 200,3490,14200,7990388343700,대한항공,운송,산업재,1.0,1.0,1.0
162,KOSPI 200,35420,62600,30435370515600,NAVER,미디어,정보기술,2.0,1.0,1.5
140,Merged,5380,11100,49286840898000,현대차,산업,산업재,2.0,1.0,1.5
...,...,...,...,...,...,...,...,...,...,...
197,KOSPI 200,114090,90,795463916200,GKL,금융,금융,29.0,28.0,28.5
128,KOSPI 200,280360,554,1152904942800,롯데웰푸드,소비재,생활소비재,28.0,30.0,29.0
133,KOSPI 200,19170,947,821797194900,신풍제약,소비재,생활소비재,25.0,35.0,30.0
129,KOSPI 200,161890,300,1099996588200,한국콜마,소비재,생활소비재,31.0,31.0,31.0


In [77]:
import pandas as pd

df = df7

# 가중치 설정
시총순위_가중치 = 0.5
기사발생량순위_가중치 = 0.5

# 가중평균 순위 계산
df['가중평균순위'] = (df['산업군내시총순위'] * 시총순위_가중치 +
                 df['산업군내기사발생량순위'] * 기사발생량순위_가중치)

# 새로운 순위에 따라 DataFrame 정렬
df_sorted = df.sort_values('가중평균순위').reset_index(drop=True)

# 가중평균순위와 기존 순위와의 차이 계산
df_sorted['시총순위대비값'] = df_sorted['가중평균순위'] - df_sorted['산업군내시총순위']
df_sorted['기사발생량순위대비값'] = df_sorted['가중평균순위'] - df_sorted['산업군내기사발생량순위']

df_sorted


,merged,종목코드,article,fundment,company,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위,시총순위대비값,기사발생량순위대비값
0,KOSPI 200,3550,1680000,13763836887500,LG,지주사,정보기술,1.0,1.0,1.0,0.0,0.0
1,KOSPI 200,720,39900,3674740245000,현대건설,건설,건설,1.0,1.0,1.0,0.0,0.0
2,KOSPI 200,3490,14200,7990388343700,대한항공,운송,산업재,1.0,1.0,1.0,0.0,0.0
3,KOSPI 200,35420,62600,30435370515600,NAVER,미디어,정보기술,2.0,1.0,1.5,0.5,-0.5
4,Merged,5380,11100,49286840898000,현대차,산업,산업재,2.0,1.0,1.5,0.5,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...
205,KOSPI 200,114090,90,795463916200,GKL,금융,금융,29.0,28.0,28.5,0.5,-0.5
206,KOSPI 200,280360,554,1152904942800,롯데웰푸드,소비재,생활소비재,28.0,30.0,29.0,-1.0,1.0
207,KOSPI 200,19170,947,821797194900,신풍제약,소비재,생활소비재,25.0,35.0,30.0,-5.0,5.0
208,KOSPI 200,161890,300,1099996588200,한국콜마,소비재,생활소비재,31.0,31.0,31.0,0.0,0.0


In [78]:
# 시총순위대비값과 기사발생량순위대비값의 괴리율 계산
df_sorted['괴리율'] = abs(df_sorted['시총순위대비값'] - df_sorted['기사발생량순위대비값'])

# 괴리율이 가장 큰 기업 순으로 정렬
df_sorted_by_discrepancy = df_sorted.sort_values('괴리율', ascending=False)

# 괴리율이 가장 큰 기업과 해당 수치 출력
df_sorted_by_discrepancy.head(30)


,merged,종목코드,article,fundment,company,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위,시총순위대비값,기사발생량순위대비값,괴리율
176,KOSPI 200,1680,75200,672171685000,대상,소비재,산업재,1.0,37.0,19.0,-18.0,18.0,36.0
175,KOSPI 200,21240,300,4132778664000,코웨이,소비재,생활소비재,31.0,7.0,19.0,12.0,-12.0,24.0
145,Legacy,17940,36600,457562000000,E1,에너지/화학,정보기술,2.0,26.0,14.0,-12.0,12.0,24.0
190,KOSPI 200,670,3030,869442880000,영풍,소비재,생활소비재,11.0,34.0,22.5,-11.5,11.5,23.0
182,KOSPI 200,383220,253,2692987372500,F&F,소비재,생활소비재,32.0,10.0,21.0,11.0,-11.0,22.0
154,KOSPI 200,138040,355,16615501713800,메리츠금융지주,금융,금융,25.0,5.0,15.0,10.0,-10.0,20.0
142,KOSPI 200,90430,1320,7101020942600,아모레퍼시픽,소비재,생활소비재,23.0,4.0,13.5,9.5,-9.5,19.0
201,KOSPI 200,32350,1780,745714119370,롯데관광개발,소비재,경기소비재,17.0,36.0,26.5,-9.5,9.5,19.0
186,KOSPI 200,2790,300,2193387588000,아모레G,소비재,생활소비재,31.0,13.0,22.0,9.0,-9.0,18.0
193,KOSPI 200,120110,1920,1029214003400,코오롱인더,소비재,생활소비재,15.0,32.0,23.5,-8.5,8.5,17.0


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
df = df_sorted

# 정보기술 산업군에 해당하는 데이터만 필터링
df_it = df[df['대분류'] == '금융'].sort_values('가중평균순위')

# 가중평균순위로 정렬
df_it_sorted = df_it.sort_values('가중평균순위')

# 시각화
fig, ax = plt.subplots(figsize=(10, 6))

# 막대그래프의 위치를 결정하기 위한 인덱스
index = np.arange(len(df_it_sorted))

# 막대그래프 폭
bar_width = 0.35

# 시총순위대비값 막대그래프
bar1 = ax.bar(index, df_it_sorted['시총순위대비값'], bar_width, label='시총순위대비값', color='b')

# 기사발생량순위대비값 막대그래프
bar2 = ax.bar(index + bar_width, df_it_sorted['기사발생량순위대비값'], bar_width, label='기사발생량순위대비값', color='g')

# 제목 및 레이블 추가
ax.set_xlabel('Company')
ax.set_ylabel('Values')
ax.set_title('Information Technology Companies: Market Cap vs Article Frequency Ranking Difference')
ax.set_xticks(index + bar_width / 2)
ax.set_xticklabels(df_it_sorted['company'])

# 범례 추가
ax.legend()

# 그래프 표시
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [82]:
# 각 산업군에서 순위별로 최대 7위까지의 기업을 나열
ranked_dfs = []  # 최종 결과를 저장할 리스트

for industry, group in industry_groups:
    # 순위별 기업 리스트 생성
    article_ranked_companies = group.sort_values('산업군내기사발생량순위')['company'].head(7).tolist()
    cap_ranked_companies = group.sort_values('산업군내시총순위')['company'].head(7).tolist()
    avg_ranked_companies = group.sort_values('가중평균순위')['company'].head(7).tolist()

    # 리스트 길이 일치시키기
    max_length = 7
    article_ranked_companies += [''] * (max_length - len(article_ranked_companies))
    cap_ranked_companies += [''] * (max_length - len(cap_ranked_companies))
    avg_ranked_companies += [''] * (max_length - len(avg_ranked_companies))

    # 새로운 DataFrame 생성
    ranked_df = pd.DataFrame({
        f'{industry}_기사발생량순위': article_ranked_companies,
        f'{industry}_시총순위': cap_ranked_companies,
        f'{industry}_가중평균순위': avg_ranked_companies
    })

    ranked_dfs.append((industry, ranked_df))  # 결과 저장

# 각 산업군별 결과 출력
for industry, df in ranked_dfs:
    print(f"산업군: {industry}\n", df, "\n")

산업군: 건설
   건설_기사발생량순위    건설_시총순위  건설_가중평균순위
0       현대건설       현대건설       현대건설
1      쌍용C&E      쌍용C&E      쌍용C&E
2        KCC        KCC        KCC
3       GS건설  HD현대인프라코어  HD현대인프라코어
4       대우건설    현대엘리베이터       대우건설
5         풍산       대우건설       GS건설
6  HD현대인프라코어         풍산    현대엘리베이터 

산업군: 금융
   금융_기사발생량순위  금융_시총순위 금융_가중평균순위
0     DB손해보험     KB금융      KB금융
1       KB금융     신한지주    DB손해보험
2       기업은행     삼성생명      기업은행
3       삼성증권   하나금융지주      삼성생명
4       현대해상  메리츠금융지주    하나금융지주
5       삼성카드     삼성화재      신한지주
6      한화시스템    카카오뱅크     카카오뱅크 

산업군: 미디어
   미디어_기사발생량순위 미디어_시총순위 미디어_가중평균순위
0         카카오    NAVER      NAVER
1       NAVER      카카오        카카오
2        제일기획     크래프톤        하이브
3        하나투어      하이브       제일기획
4         하이브      넷마블       크래프톤
5         넷마블     제일기획        넷마블
6        크래프톤     하나투어       하나투어 

산업군: 산업
     산업_기사발생량순위 산업_시총순위    산업_가중평균순위
0           LS     현대차          현대차
1          현대차      기아          HMM
2          HMM     HMM           기아
3    

In [83]:
df_sorted_by_discrepancy.to_csv('/content/KOSPI200_searchtime_and_fundmenmtal_updated0401_2.csv', encoding = 'euc-kr', index=False)

## 최종 취급대상 후보 산출

In [2]:
# CSV 파일 로드
import pandas as pd
file_path = '/content/KOSPI200_searchtime_and_fundmenmtal_updated0401_2.csv'
df8 = pd.read_csv(file_path, encoding = 'euc-kr')

In [ ]:
industry_groups = df8.groupby('대분류')

In [7]:
df8.head()

,merged,종목코드,article,fundment,company,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위,시총순위대비값,기사발생량순위대비값,괴리율
0,KOSPI 200,1680,75200,672171685000,대상,소비재,산업재,1.0,37.0,19.0,-18.0,18.0,36.0
1,KOSPI 200,21240,300,4132778664000,코웨이,소비재,생활소비재,31.0,7.0,19.0,12.0,-12.0,24.0
2,Legacy,17940,36600,457562000000,E1,에너지/화학,정보기술,2.0,26.0,14.0,-12.0,12.0,24.0
3,KOSPI 200,670,3030,869442880000,영풍,소비재,생활소비재,11.0,34.0,22.5,-11.5,11.5,23.0
4,KOSPI 200,383220,253,2692987372500,F&F,소비재,생활소비재,32.0,10.0,21.0,11.0,-11.0,22.0


In [8]:
df8['기사발생량순위'] = df8['article'].rank(method='dense', ascending=False)
df8['시총순위'] = df8['fundment'].rank(method='dense', ascending=False)

In [9]:
df8.head()

,merged,종목코드,article,fundment,company,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위,시총순위대비값,기사발생량순위대비값,괴리율,기사발생량순위,시총순위
0,KOSPI 200,1680,75200,672171685000,대상,소비재,산업재,1.0,37.0,19.0,-18.0,18.0,36.0,7.0,202.0
1,KOSPI 200,21240,300,4132778664000,코웨이,소비재,생활소비재,31.0,7.0,19.0,12.0,-12.0,24.0,145.0,81.0
2,Legacy,17940,36600,457562000000,E1,에너지/화학,정보기술,2.0,26.0,14.0,-12.0,12.0,24.0,14.0,206.0
3,KOSPI 200,670,3030,869442880000,영풍,소비재,생활소비재,11.0,34.0,22.5,-11.5,11.5,23.0,74.0,187.0
4,KOSPI 200,383220,253,2692987372500,F&F,소비재,생활소비재,32.0,10.0,21.0,11.0,-11.0,22.0,150.0,101.0


In [12]:
import pandas as pd

# 새로운 데이터프레임 생성을 위한 빈 리스트
new_data = []

# 대분류별로 순회
for category in df8['대분류'].unique():
    # 해당 대분류에서 가중평균 순위 1위부터 7위까지 필터링
    filtered_df = df8[df8['대분류'] == category].sort_values('가중평균순위').head(5) # 산업군별 5위까지
    new_data.append(filtered_df)

# 새로운 데이터프레임으로 합치기
result_df = pd.concat(new_data).reset_index(drop=True)

# 결과 출력
result_df


,merged,종목코드,article,fundment,company,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위,시총순위대비값,기사발생량순위대비값,괴리율,기사발생량순위,시총순위
0,KOSPI 200,33780,57200,12185153469500,KT&G,소비재,생활소비재,2.0,2.0,2.0,0.0,0.0,0.0,11.0,28.0
1,Merged,28260,7350,29416279695000,삼성물산,소비재,생활소비재,4.0,1.0,2.5,1.5,-1.5,3.0,36.0,12.0
2,KOSPI 200,51900,9520,5934914860000,LG생활건강,소비재,생활소비재,3.0,5.0,4.0,-1.0,1.0,2.0,32.0,59.0
3,KOSPI 200,326030,2760,7181325025000,SK바이오팜,소비재,생활소비재,12.0,3.0,7.5,4.5,-4.5,9.0,77.0,51.0
4,KOSPI 200,35250,5410,3485090745000,강원랜드,소비재,경기소비재,7.0,9.0,8.0,-1.0,1.0,2.0,48.0,94.0
5,Merged,15760,28400,14123209694000,한국전력,에너지/화학,에너지/화학,3.0,4.0,3.5,-0.5,0.5,1.0,15.0,23.0
6,Merged,10950,3060000,8837749172000,S-Oil,에너지/화학,에너지/화학,1.0,8.0,4.5,-3.5,3.5,7.0,1.0,45.0
7,KOSPI 200,51910,1940,30990038577000,LG화학,에너지/화학,에너지/화학,8.0,3.0,5.5,2.5,-2.5,5.0,89.0,10.0
8,Merged,96770,2670,11105328440000,SK이노베이션,에너지/화학,에너지/화학,6.0,5.0,5.5,0.5,-0.5,1.0,78.0,32.0
9,KOSPI 200,373220,1340,92430000000000,LG에너지솔루션,에너지/화학,에너지/화학,11.0,1.0,6.0,5.0,-5.0,10.0,104.0,3.0


In [16]:
# 새로운 데이터프레임 생성을 위한 빈 리스트
new_data2 = []

# 대분류별로 순회
for category in df8['대분류'].unique():
    # 해당 대분류에서 가중평균 순위 1위부터 7위까지 필터링
    filtered_df2 = df8[df8['대분류'] == category].sort_values('가중평균순위')
    filtered_df2 = filtered_df2[(filtered_df2['가중평균순위'] >= 6) & (filtered_df2['가중평균순위'] <= 10)]
    new_data2.append(filtered_df2)

# 새로운 데이터프레임으로 합치기
result_df2 = pd.concat(new_data2).reset_index(drop=True)

# 결과 출력
result_df2

,merged,종목코드,article,fundment,company,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위,시총순위대비값,기사발생량순위대비값,괴리율,기사발생량순위,시총순위
0,KOSPI 200,326030,2760,7181325025000,SK바이오팜,소비재,생활소비재,12.0,3.0,7.5,4.5,-4.5,9.0,77.0,51.0
1,KOSPI 200,35250,5410,3485090745000,강원랜드,소비재,경기소비재,7.0,9.0,8.0,-1.0,1.0,2.0,48.0,94.0
2,KOSPI 200,8770,4650,2354887260000,호텔신라,소비재,생활소비재,8.0,11.0,9.5,-1.5,1.5,3.0,55.0,111.0
3,KOSPI 200,373220,1340,92430000000000,LG에너지솔루션,에너지/화학,에너지/화학,11.0,1.0,6.0,5.0,-5.0,10.0,104.0,3.0
4,KOSPI 200,6400,1370,32388093630000,삼성SDI,에너지/화학,산업재,10.0,2.0,6.0,4.0,-4.0,8.0,103.0,9.0
5,KOSPI 200,9830,3830,4873153395600,한화솔루션,에너지/화학,에너지/화학,5.0,10.0,7.5,-2.5,2.5,5.0,65.0,69.0
6,KOSPI 200,34020,1800,11030462934120,두산에너빌리티,에너지/화학,에너지/화학,9.0,6.0,7.5,1.5,-1.5,3.0,92.0,33.0
7,Merged,36460,5300,2478604050000,한국가스공사,에너지/화학,에너지/화학,4.0,13.0,8.5,-4.5,4.5,9.0,51.0,108.0
8,KOSPI 200,11170,1340,5103107486700,롯데케미칼,에너지/화학,에너지/화학,11.0,9.0,10.0,1.0,-1.0,2.0,104.0,66.0
9,KOSPI 200,24110,25000,10765249231500,기업은행,금융,금융,3.0,9.0,6.0,-3.0,3.0,6.0,18.0,37.0


## 전처리 후

In [10]:
import pandas as pd

file_path = '/content/KOSPI200_updated0402_sorted.csv'

#df8.to_csv(file_path, encoding = 'euc-kr', index=False)
df8 = pd.read_csv(file_path, encoding = 'euc-kr')

In [11]:
df8.head()

,merged,종목코드,article,fundment,company,시총순위,기사발생량순위,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위,시총순위대비값,기사발생량순위대비값,괴리율
0,Merged,5930,26500,4.920000e+14,삼성전자,1,17,전자,정보기술,2,1,1.5,0.5,-0.5,1
1,Merged,660,5620,1.330000e+14,SK하이닉스,2,44,전자,정보기술,7,2,4.5,2.5,-2.5,5
2,KOSPI 200,373220,"2,890",9.240000e+13,LG에너지솔루션,3,104,에너지,에너지/화학,11,1,6.0,5.0,-5.0,10
3,KOSPI 200,207940,685,5.910000e+13,삼성바이오로직스,4,126,헬스케어,헬스케어,7,1,4.0,3.0,-3.0,6
4,Merged,5380,11100,4.930000e+13,현대차,5,29,차량,산업재,2,1,1.5,0.5,-0.5,1


In [12]:
unique_ranks_per_industry = df8.groupby('대분류')['company'].nunique()
unique_ranks_per_industry

대분류
건설       13
금융       17
미디어       8
생활소비재    19
소재       16
에너지      15
운송        7
유통       10
전자        9
정보기술      6
중공업      14
지주사      17
차량       11
헬스케어     14
화학       19
Name: company, dtype: int64

In [21]:
# 각 산업군('industry') 내에서 'count' 컬럼에 따라 순위를 매기고, 결과를 새로운 컬럼에 저장합니다.
df4 = df8
df4 ['산업군내기사발생량순위'] = df4.groupby('대분류')['article'].rank(method='dense', ascending=False)
df4 ['산업군내시총순위'] = df4.groupby('대분류')['fundment'].rank(method='dense', ascending=False)

# 가중치 설정
시총순위_가중치 = 0.75
기사발생량순위_가중치 = 0.25

df7 = df4
# 가중평균 계산
df7['가중평균순위'] = (df7['산업군내시총순위'] * 시총순위_가중치 +
                 df7['산업군내기사발생량순위'] * 기사발생량순위_가중치)

df7['가중평균순위의순위'] = df7.groupby('대분류')['가중평균순위'].rank(method='dense', ascending=True)

# 새로운 순위에 따라 DataFrame 정렬 (선택적)
df7 = df7.sort_values('가중평균순위의순위')
df7

,merged,종목코드,article,fundment,company,시총순위,기사발생량순위,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위,시총순위대비값,기사발생량순위대비값,괴리율,가중평균순위의순위
7,KOSPI 200,5490,857,3.570000e+13,POSCO홀딩스,8,120,지주사,철강/소재,2.0,1.0,1.25,2.5,-2.5,5,1.0
37,KOSPI 200,329180,3340,1.050000e+13,HD현대중공업,38,69,중공업,중공업,6.0,1.0,2.25,1.0,-1.0,2,1.0
6,KOSPI 200,68270,901,4.170000e+13,셀트리온,7,117,헬스케어,생명과학 healthcare,2.0,2.0,2.00,2.0,-2.0,4,1.0
29,KOSPI 200,17670,6940,1.130000e+13,SK텔레콤,30,37,정보기술,정보기술,1.0,2.0,1.75,0.5,-0.5,1,1.0
27,KOSPI 200,33780,57200,1.220000e+13,KT&G,28,11,생활소비재,생활소비재,3.0,1.0,1.50,0.0,0.0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
159,KOSPI 200,5300,1320,1.180000e+12,롯데칠성,171,106,생활소비재,생활소비재,15.0,14.0,14.25,-2.0,2.0,4,16.0
91,KOSPI 200,16360,24600,3.580000e+12,삼성증권,92,19,금융,금융,13.0,15.0,14.50,-6.0,6.0,12,16.0
168,KOSPI 200,120110,1920,1.030000e+12,코오롱인더,181,90,생활소비재,생활소비재,10.0,17.0,15.25,-8.5,8.5,17,17.0
172,KOSPI 200,1800,300,8.770000e+11,오리온홀딩스,185,145,생활소비재,생활소비재,7.0,18.0,15.25,-1.0,1.0,2,17.0


In [22]:
df8 = df7
# 새로운 데이터프레임 생성을 위한 빈 리스트
new_data = []

# 대분류별로 순회
for category in df8['대분류'].unique():
    # 해당 대분류에서 가중평균 순위 1위부터 7위까지 필터링
    filtered_df = df8[df8['대분류'] == category].sort_values('가중평균순위의순위').head(7) # 산업군별 5위까지
    new_data.append(filtered_df)

# 새로운 데이터프레임으로 합치기
result_df = pd.concat(new_data).reset_index(drop=True)

# 결과 출력
result_df

,merged,종목코드,article,fundment,company,시총순위,기사발생량순위,대분류,초기분류,산업군내기사발생량순위,산업군내시총순위,가중평균순위,시총순위대비값,기사발생량순위대비값,괴리율,가중평균순위의순위
0,KOSPI 200,5490,857,3.570000e+13,POSCO홀딩스,8,120,지주사,철강/소재,2.0,1.0,1.25,2.5,-2.5,5,1.0
1,KOSPI 200,78930,97400,4.480000e+12,GS,73,6,지주사,금융,1.0,5.0,4.00,0.0,0.0,0,2.0
2,KOSPI 200,402340,4610,1.100000e+13,SK스퀘어,34,56,지주사,정보기술,5.0,4.0,4.25,0.5,-0.5,1,3.0
3,KOSPI 200,3550,1680000,1.380000e+13,LG,24,2,지주사,정보기술,12.0,2.0,4.50,0.0,0.0,0,4.0
4,KOSPI 200,34730,1580000,1.310000e+13,SK,26,3,지주사,산업재,13.0,3.0,5.50,0.0,0.0,0,5.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99,Merged,5380,11100,4.930000e+13,현대차,5,29,차량,산업재,8.0,1.0,2.75,0.5,-0.5,1,3.0
100,KOSPI 200,18880,300,3.130000e+12,한온시스템,97,145,차량,산업재,4.0,4.0,4.00,3.0,-3.0,6,4.0
101,KOSPI 200,73240,1580,1.690000e+12,금호타이어,133,97,차량,산업재,7.0,5.0,5.50,-1.0,1.0,2,5.0
102,KOSPI 200,204320,300,1.560000e+12,HL만도,140,145,차량,산업재,4.0,6.0,5.50,-2.0,2.0,4,5.0


In [25]:
#result_df(['종목코드','company','fundment','시총순위','대분류','기사발생량순위',])

In [39]:
import pandas as pd

file_path = '/content/크롤링 재분배.csv'

#df8.to_csv(file_path, encoding = 'euc-kr', index=False)
df8 = pd.read_csv(file_path, encoding = 'euc-kr')

In [40]:
uniqued = df8.groupby('대분류')['종목명'].nunique()
uniqued

대분류
건설       6
금융       6
미디어      6
생활소비재    6
소재       6
에너지      7
운송       7
유통       7
전자       7
정보기술     6
중공업      6
지주사      6
차량       6
헬스케어     6
화학       6
Name: 종목명, dtype: int64

In [41]:
'''data = {
    '대분류': np.random.choice(['금융', 'IT', '제조', '서비스'], size=100),
    '기업명': [f'기업_{i}' for i in range(1, 101)],
    '기사발생량': np.random.randint(1, 1000, size=100)
}'''

"data = {\n    '대분류': np.random.choice(['금융', 'IT', '제조', '서비스'], size=100),\n    '기업명': [f'기업_{i}' for i in range(1, 101)],\n    '기사발생량': np.random.randint(1, 1000, size=100)\n}"

In [42]:
import pandas as pd
import numpy as np

# 기사 수집량에 따라 내림차순으로 정렬
df8 = df8.sort_values(by='기사발생량', ascending=False).reset_index(drop=True)
df8.head()

,종목코드,대분류,종목명,완료,열1,가중평균순위의순위,시총,시총순위,기사발생량,기사발생량순위
0,10950,화학,S-Oil,나연,완료,1,8.840000e+12,45,306000,1
1,35720,미디어,카카오,지헌,완료,3,2.390000e+13,14,109000,4
2,78930,지주사,GS,NaN,NaN,1,4.480000e+12,73,97400,6
3,1040,지주사,CJ,NaN,NaN,2,3.790000e+12,87,71800,8
4,35420,미디어,NAVER,NaN,NaN,1,3.040000e+13,11,62600,10


In [58]:
df_sorted = df8.sort_values(by='기사발생량', ascending=False).reset_index(drop=True)

In [59]:
# '기사발생량' 열을 정수형으로 변환 시도
df_sorted['기사발생량'] = pd.to_numeric(df_sorted['기사발생량'], errors='coerce')

# 변환 후 데이터 유형 재확인
df_sorted['기사발생량'].dtype

dtype('int64')

In [60]:
# 이미 할당된 기업 수에 따른 초기 설정
initial_companies_assigned = {'나연': 7, '지헌': 5, '한재': 2, '해리': 1}

# 담당자별로 할당된 기사 수집량을 추적하는 딕셔너리 초기화
collector_totals = {'나연': 50000, '지헌': 40000, '한재': 5000, '해리': 0000}

In [61]:
# 담당자가 이미 할당된 기업과 할당되지 않은 기업 분리
assigned_df = df_sorted.dropna(subset=['완료'])
unassigned_df = df_sorted[df_sorted['완료'].isna()]

# 이미 할당된 기업의 수를 고려하여 담당자별로 할당된 기업 수를 추적하는 딕셔너리 초기화
company_totals = initial_companies_assigned.copy()

# 이미 할당된 담당자별 기업 수 업데이트
for collector in ['나연','지헌','한재','해리']:
    company_totals[collector] = assigned_df[assigned_df['완료'] == collector].shape[0]

# 남은 기업들에 대한 담당자 분배 로직 적용
collectors_updated = []

for _, row in unassigned_df.iterrows():
    # 조건 1: 가장 적은 기업을 맡고 있는 담당자 선택
    # 조건 2: 같은 수의 기업을 맡고 있다면, 기사 수집량이 가장 적은 담당자 선택
    min_collector = min(company_totals, key=lambda x: (company_totals[x], collector_totals[x]))

    # 해당 담당자에게 기업 할당
    collectors_updated.append(min_collector)

    # 할당된 담당자의 총 기업 수와 기사 수집량 업데이트
    company_totals[min_collector] += 1
    collector_totals[min_collector] += row['기사발생량']

# 할당된 담당자 정보를 미할당된 기업 데이터프레임에 추가
unassigned_df['완료'] = collectors_updated

# 미할당된 기업 데이터프레임과 이미 할당된 기업 데이터프레임 합치기
final_df = pd.concat([assigned_df, unassigned_df]).sort_index()

# 최종 할당된 담당자별 기업 수 및 기사 수집량 확인

final_company_totals = pd.Series(company_totals)
print('할당종목수')
print(final_company_totals)

final_article_totals = pd.Series(collector_totals)
print('할당기사수(이미 한 거 제외)')
print(final_article_totals)

할당종목수
나연    24
지헌    24
한재    23
해리    23
dtype: int64
할당기사수(이미 한 거 제외)
나연     92767
지헌    118195
한재    244599
해리    300742
dtype: int64


<ipython-input-61-b3bf7379309b>:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  unassigned_df['완료'] = collectors_updated


In [62]:
final_df

,종목코드,대분류,종목명,완료,열1,가중평균순위의순위,시총,시총순위,기사발생량,기사발생량순위
0,10950,화학,S-Oil,나연,완료,1,8.840000e+12,45,306000,1
1,35720,미디어,카카오,지헌,완료,3,2.390000e+13,14,109000,4
2,78930,지주사,GS,해리,NaN,1,4.480000e+12,73,97400,6
3,1040,지주사,CJ,한재,NaN,2,3.790000e+12,87,71800,8
4,35420,미디어,NAVER,해리,NaN,1,3.040000e+13,11,62600,10
...,...,...,...,...,...,...,...,...,...,...
89,6840,지주사,AK홀딩스,지헌,NaN,6,2.000000e+11,210,300,145
90,120,운송,CJ대한통운,한재,NaN,4,2.620000e+12,103,300,145
91,1570,화학,금양,해리,NaN,3,6.710000e+12,54,300,145
92,361610,전자,SK아이이테크놀로지,나연,NaN,4,5.220000e+12,64,300,145


In [64]:
final_df.to_csv('크롤링 재분배_자동분배기.csv', encoding = 'euc-kr', index=False)